# Replicate experiments with new skew-spectrums

( This is done using my sage310 environment )

In [1]:
import pandas as pd
import scipy
import scipy.io
import pickle
import numpy as np
from scipy.spatial.distance import pdist, squareform
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#import tensorflow as tf
import time
from sklearn.decomposition  import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

from spectrum_utils import *
import numpy as np



import warnings
warnings.simplefilter('ignore')


augmentation_factor = 0  # how much did we augment the dataset? (this is our NeurIPS trick)
num_atoms = 23
verbose=True
y_scaling_factor = 2000.

rand_state = 42
#tf.random.set_seed(rand_state)
np.random.seed(rand_state)


# hyperparameters for classification
total_range = 1787.119995
num_bins = 10




qm7 = scipy.io.loadmat('/Users/scinawa/workspace/grouptheoretical/hpctmp/t0926977/multi_orbit_bispectrum/qm7.mat')
R = qm7['R']

y = np.transpose(qm7['T']).reshape((7165,))
y_scaling_factor = 2000.
y_scaled = y / y_scaling_factor



directory='/hpctmp/t0926977/multi_orbit_bispectrum/qm7-skew_spectrums/'

# file_1full = '1orbit_qm7_full_skew_spectrum.npy'
# file_2full = '2orbit_qm7_full_skew_spectrum.npy'

# file_1redu = '1orbit_qm7_reduced_skew_spectrum.npy'
# file_2redu = '2orbit_qm7_reduced_skew_spectrum.npy'

# file_1redu3 = "1orbit_qm7_reduced_3skew_spectrum.npy"
# file_2redu3 = "2orbit_qm7_reduced_3skew_spectrum.npy"

In [2]:

def mean_dist(x):
    x[x == 0] = np.nan
    return np.nanmean(x, axis=0)


def extract_dataset():
    iu = np.triu_indices(num_atoms,k=0) 
    iu_dist = np.triu_indices(num_atoms,k=1) # for the pairwise distance matrix, all diagonol entries will be 0 
    CM = np.zeros((qm7['X'].shape[0], num_atoms*(num_atoms+1)//2), dtype=float)
    eigs = np.zeros((qm7['X'].shape[0], num_atoms), dtype=float)
    centralities = np.zeros((qm7['X'].shape[0], num_atoms), dtype=float)
    interatomic_dist = np.zeros((qm7['X'].shape[0], ((num_atoms*num_atoms)-num_atoms)//2), dtype=float) 


    for i, cm in enumerate(qm7['X']):
        coulomb_vector = cm[iu]
        # Sort elements by decreasing order
        shuffle = np.argsort(-coulomb_vector)
        CM[i] = coulomb_vector[shuffle]
        dist = squareform(pdist(R[i]))
        # we can extract the upper triangle of the distance matri: return vector of dimension (1,num_atoms)
        dist_vector = dist[iu_dist]
        shuffle = np.argsort(-dist_vector)
        interatomic_dist[i] = dist_vector[shuffle]
        
        w,v = np.linalg.eig((dist))
        eigs[i] = w[np.argsort(-w)]
        centralities[i] = np.array(list(nx.eigenvector_centrality(nx.Graph(dist)).values()))
        
        if verbose and i % 500 == 0:
            print("Processed {} molecules".format(i))
        
    mean_dists = np.apply_along_axis(mean_dist, axis=1, arr=interatomic_dist)

    return qm7['X'], CM, eigs, centralities, interatomic_dist, mean_dists

Create the dictionary of feature that will be used for machine learning.

In [3]:
qm7['X'], CM, eigs, centralities, interatomic_dist, mean_dists = extract_dataset()
megadataset= {'qm7': qm7['X'], 'CM' : CM, 'eigs': eigs, 'centralities': centralities, 
                   'interatomic_dist': interatomic_dist, 'mean_dist': mean_dists }

Processed 0 molecules
Processed 500 molecules
Processed 1000 molecules
Processed 1500 molecules
Processed 2000 molecules
Processed 2500 molecules
Processed 3000 molecules
Processed 3500 molecules
Processed 4000 molecules
Processed 4500 molecules
Processed 5000 molecules
Processed 5500 molecules
Processed 6000 molecules
Processed 6500 molecules
Processed 7000 molecules


In [5]:
for k in range(4,6):
	new_kcor1orb = []
	new_kcor2orb = []
	print("kreating {}-th correlation".format(k))
	for i in range(len(qm7['X'])):

		func_1o = create_func_on_group_from_matrix_1orbit(qm7['X'][i].reshape(23,23))
		func_2o = create_func_on_group_from_matrix_2orbits(qm7['X'][i].reshape(23,23))


		new_kcor1orb.append(reduced_k_correlation(func_1o, k=k, method="extremedyn", vector=True))     # vector = False !!!!!
		new_kcor2orb.append(reduced_k_correlation(func_2o, k=k, method="extremedyn", vector=True))

	megadataset["1-{}-corre".format(k)] = new_kcor1orb
	megadataset["2-{}-corre".format(k)] = new_kcor2orb

############ USE WITH CARE!!!!!! IT WILL OVERWRITE THE PERVIOUSLY COMPUTED FEATURES FILE
with open('megadump-all-features.pickle', 'wb') as handle:
   pickle.dump(megadataset, handle, protocol=pickle.HIGHEST_PROTOCOL)


kreating 4-th correlation
kreating 5-th correlation


In [4]:
handle = open('megadump-all-features.pickle', 'rb')
megadataset = pickle.load(handle)

### Definition of machine learning models
This code is for the machine learning models. 

In [5]:
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.linear_model import BayesianRidge
#from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from xgboost.sklearn import XGBRegressor
#from lightgbm import LGBMRegressor



models = {'GradientBR' :  GradientBoostingRegressor, 'Elastic' : ElasticNet, 
          #'SGD' : SGDRegressor, 
          # 'SVR': SVR, 
          #'Bayesian': BayesianRidge,
          #'CatBoost': CatBoostRegressor, 
          #'KernelRidge': KernelRidge, 
          'Linear': LinearRegression, 'XGBR': XGBRegressor } #'LGMR': LGBMRegressor}


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import warnings
import sys
import pickle
warnings.simplefilter('ignore')
from sklearn.decomposition import PCA


def machine_learning_models(X_train, y_train, X_test, y_test):


    for model_name,model in models.items():
        print("Working with ", model_name, ". Error: ", end="")
        #if model_name == "SGD":
        #    import pdb
        #    pdb.set_trace()
        #    clf = model()
        clf = model()
        clf.fit(X_train,y_train)
        y_pred = clf.predict(X_test)
        print("(test)", mean_absolute_error(y_pred , y_test)) 



def machine_learning_xgboost(X_train, y_train, X_test, y_test, X_val, y_val):

    start_time = time.time()

    # n_folds = 5
    early_stopping = 50

    # xg_train = xgb.DMatrix(X_train, label=y_train)
    num_iters = 300
    params = {"objective":"reg:squarederror",    # {"objective":"reg:linear", 
            'booster': 'gbtree', 
            'eval_metric': 'mae',
            'subsample': 0.9,
            'colsample_bytree':0.2,
            'learning_rate': 0.05,
            'max_depth': 6, 
            'reg_lambda': .9, 
            'reg_alpha': .01,
            'seed': rand_state}

    # cv = xgb.cv(params,
    #             xg_train, 
    #             num_boost_round=num_iters, 
    #             nfold=n_folds, 
    #             early_stopping_rounds=early_stopping, 
    #             verbose_eval = 0, 
    #             seed=rand_state,
    #             as_pandas=False)


    # plt.figure(figsize=(8,8))
    # plt.plot(cv['train-mae-mean'][100:], label='Train loss: ' + str(np.min(cv['train-mae-mean'])))
    # plt.plot(cv['test-mae-mean'][100:], label='Test loss: ' + str(np.min(cv['test-mae-mean'])))
    # plt.legend()
    # plt.xlabel('Epoch')
    # plt.ylabel('Mean absolute error')
    # plt.savefig("verybad.png")

    model_xgb = xgb.XGBRegressor(**params, random_state=rand_state, n_estimators=num_iters)
    model_xgb.fit(X_train, y_train, 
                early_stopping_rounds=early_stopping, 
                #eval_metric='mae', 
                eval_set=[(X_val, y_val)], 
                verbose=False)


    y_train_pred = model_xgb.predict(X_train)
    print('Train mean absoulte error: ', mean_absolute_error(y_train, y_train_pred))

    y_val_pred = model_xgb.predict(X_val)
    print('Validation mean absoulte error: ', mean_absolute_error(y_val, y_val_pred))


    y_test_pred = model_xgb.predict(X_test)
    print('TEST mean absoulte error: ', mean_absolute_error(y_test, y_test_pred))


    
    print("--- %s seconds ---" % (time.time() - start_time))

### Machine Learning workbench

In the code below we can play with experiments, by changing the feature in the dataset and running the experiments that we want.

In [6]:
def PCA_everything(dataset, ncomponents):
    pca = PCA(n_components= ncomponents)
    return pca.fit_transform(dataset)
print(megadataset.keys())

dict_keys(['qm7', 'CM', 'eigs', 'centralities', 'interatomic_dist', 'mean_dist', '1-2-corre', '2-2-corre', '1-3-corre', '2-3-corre', '1-4-corre', '2-4-corre', '1-5-corre', '2-5-corre'])


In [27]:
print("In this experiemnt we do machine learning only with the k-correlations for all the ks, and all the orbits.")
for orbit, k in [(orbit, k) for orbit in range(1,3) for k in range(2, 6) ]:
    print("{}-{}-corre".format(orbit,k))
    X = np.concatenate((megadataset['CM'], megadataset['eigs'], PCA_everything(megadataset["{}-{}-corre".format(orbit,k)], ncomponents=20) ), axis=1)
    
    # with cross validation no need to further split the data. if not using cross validation you should do one...
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                test_size=0.20, 
                                                random_state=rand_state)

    machine_learning_models(X_train, y_train, X_test, y_test)

In this experiemnt we do machine learning only with the k-correlations for all the ks, and all the orbits.
1-2-corre
Working with:  GradientBR Error: TEST 10.941690427068586
Working with:  Elastic Error: TEST 28.242899458574186
Working with:  Linear Error: TEST 9620.899963108961
Working with:  XGBR Error: TEST 8.267371
1-3-corre
Working with:  GradientBR Error: TEST 10.868614833002184
Working with:  Elastic Error: TEST 28.660986459607603
Working with:  Linear Error: TEST 21.342878473041843
Working with:  XGBR Error: TEST 8.470072
1-4-corre
Working with:  GradientBR Error: TEST 10.82198118300598
Working with:  Elastic Error: TEST 27.883955706340625
Working with:  Linear Error: TEST 21.834101490299084
Working with:  XGBR Error: TEST 8.030998
2-2-corre
Working with:  GradientBR Error: TEST 10.64711116646851
Working with:  Elastic Error: TEST 28.097465672239032
Working with:  Linear Error: TEST 19.192725609970584
Working with:  XGBR Error: TEST 8.209346
2-3-corre
Working with:  GradientBR 

Let's compare 2,3 correlations together, by putting only this in the dataset.

In [8]:
print("In this experiemnt we do machine learning only with the k-correlations for all the ks, and all the orbits.")
for orbit, k in [(orbit, k) for orbit in range(1,3) for k in range(2, 6) ]:
    #print("%sorbit-%s-corre")
    print ("orbit", orbit, "k", k)
    X = megadataset["{0}-{1}-corre".format(orbit,k)]
    # with cross validation no need to further split the data. if not using cross validation you should do one...
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                test_size=0.20, 
                                                random_state=rand_state)

    machine_learning_models(X_train, y_train, X_test, y_test)

In this experiemnt we do machine learning only with the k-correlations for all the ks, and all the orbits.
Working with  GradientBR . Error: (test) 36.89754080618257
Working with  Elastic . Error: (test) 112.3235028317343
Working with  Linear . Error: (test) 61.087242
Working with  XGBR . Error: (test) 28.977999
Working with  GradientBR . Error: (test) 42.26125282194708
Working with  Elastic . Error: (test) 83.3554355955342
Working with  Linear . Error: (test) 66.10057
Working with  XGBR . Error: (test) 32.20713
Working with  GradientBR . Error: 

Let's compare 2,3 correlations together, by putting only this in the dataset. This time we also do some PCA to reduce the dimensionality of the things. 


In [ ]:
print("In this experiemnt we do machine learning only with the k-correlations for all the ks, and all the orbits.")
for orbit, k in [(orbit, k) for orbit in range(1,3) for k in range(2, 5) ]:
    print("%sorbit-%s-corre")
    X = megadataset["%s-%s-corre".format(orbit,k)]
    # with cross validation no need to further split the data. if not using cross validation you should do one...
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                test_size=0.20, 
                                                random_state=rand_state)

    machine_learning_models(X_train, y_train, X_test, y_test)


In [ ]:
    #X = skspecqm7 
    #X = megadataset['CM']
    X = megadataset["2-2-corre"]

    #X = np.concatenate((megadataset['CM'], megadataset['eigs'], skspecqm7), axis=1)

    # 2 full
    #Dev mean absoulte error:  6.7378244
    #Validation mean absoulte error:  6.7369103

    X = np.concatenate((megadataset['CM'], megadataset['eigs'], megadataset['2-3-corre']), axis=1)
    #X = np.concatenate((megadataset['CM'], megadataset['eigs'], megadataset['2-2-corre']), axis=1)
    # 1 full
    # Dev mean absoulte error:  7.145836
    # Validation mean absoulte error:  6.9577074

    #X = np.concatenate((skspecqm7, megadataset['CM'], megadataset['eigs'], megadataset['centralities']), axis=1)
    # no skew_spectrum
    #Dev mean absoulte error:  6.9068303
    #Validation mean absoulte error:  6.810617

    #X = skspecqm7




    # with cross validation no need to further split the data. if not using cross validation you should do one...
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.20, 
                                            random_state=rand_state)

    #X_train, X_val, y_train, y_val = train_test_split(X_2, y_2, 
    #                                               test_size=0.18, 
    #                                               random_state=rand_state)

    # try: 
    #     non_zero_idx = np.any(X_train[:, 0:len(skspecqm7[1]) ], axis=0)
    #     non_zero_idx = np.concatenate( ( non_zero_idx, np.array([1] *(X.shape[1]-skspecqm7.shape[1]) )    ))
    #     X_test=X_test[:,non_zero_idx]
    #     X_val=X_val[:,non_zero_idx]
    #     X_train=X_train[:,non_zero_idx]
    
    # except Exception:
    #     print("skipping taking only nonzero components of skewspectrum")

    #machine_learning_xgboost(X_train, y_train, X_test, y_test, X_val, y_val)
    machine_learning_models(X_train, y_train, X_test, y_test)


In [ ]:
    #X = skspecqm7 
    #X = megadataset['CM']
    X = megadataset["2-2-corre"]

    #X = np.concatenate((megadataset['CM'], megadataset['eigs'], skspecqm7), axis=1)

    # 2 full
    #Dev mean absoulte error:  6.7378244
    #Validation mean absoulte error:  6.7369103

    X = np.concatenate((megadataset['CM'], megadataset['eigs'], megadataset['2-3-corre']), axis=1)
    #X = np.concatenate((megadataset['CM'], megadataset['eigs'], megadataset['2-2-corre']), axis=1)
    # 1 full
    # Dev mean absoulte error:  7.145836
    # Validation mean absoulte error:  6.9577074

    #X = np.concatenate((skspecqm7, megadataset['CM'], megadataset['eigs'], megadataset['centralities']), axis=1)
    # no skew_spectrum
    #Dev mean absoulte error:  6.9068303
    #Validation mean absoulte error:  6.810617

    #X = skspecqm7




    # with cross validation no need to further split the data. if not using cross validation you should do one...
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                            test_size=0.20, 
                                            random_state=rand_state)

    #X_train, X_val, y_train, y_val = train_test_split(X_2, y_2, 
    #                                               test_size=0.18, 
    #                                               random_state=rand_state)

    # try: 
    #     non_zero_idx = np.any(X_train[:, 0:len(skspecqm7[1]) ], axis=0)
    #     non_zero_idx = np.concatenate( ( non_zero_idx, np.array([1] *(X.shape[1]-skspecqm7.shape[1]) )    ))
    #     X_test=X_test[:,non_zero_idx]
    #     X_val=X_val[:,non_zero_idx]
    #     X_train=X_train[:,non_zero_idx]
    
    # except Exception:
    #     print("skipping taking only nonzero components of skewspectrum")

    #machine_learning_xgboost(X_train, y_train, X_test, y_test, X_val, y_val)
    machine_learning_models(X_train, y_train, X_test, y_test)
